In [4]:
!pip install --quiet -U lancedb
!pip install --quiet gradio


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [49]:
!curl -o /tmp/img.png https://user-images.githubusercontent.com/917119/226205734-6063d87a-1ecc-45fe-85be-1dea6383a3d8.png 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12539  100 12539    0     0  44000      0 --:--:-- --:--:-- --:--:-- 44622


In [52]:
from PIL import Image
import io
image = Image.open("/tmp/img.png")
img_byte_arr = io.BytesIO()
image.save(img_byte_arr, format='PNG')
img_byte_arr = img_byte_arr.getvalue()

In [68]:
import pyarrow as pa

uri = "~/datasets"
db = lancedb.connect(uri)
tbl = db.open_table("chatbot")
data = tbl.to_lance().to_table()
data = data.append_column("image", pa.array([img_byte_arr]*len(data)))

db = lancedb.connect("~/datasets/demo")
db.create_table("diffusion", data)

LanceTable(diffusion)

In [69]:
tbl = db.open_table("diffusion")
tbl = tbl.create_fts_index(["text", "title"])

In [94]:
import duckdb
import lancedb

uri = "~/datasets/demo"
db = lancedb.connect(uri)
tbl = db.open_table("diffusion")

# Initialize CLIP model
def embed_func(vector_query):
    pass

def find_image_vectors(query):
    emb = embed_func(query)
    return _extract(tbl.search(emb).limit(9).to_df())

def find_image_keywords(query):
    return _extract(tbl.search(query).limit(9).to_df())

def find_image_sql(query):
    diffusiondb = tbl.to_lance()
    return _extract(duckdb.query(query).to_df())

def _extract(df):
    image_col = "image"
    label_col = "text"
    return [(PIL.Image.open(io.BytesIO(row[image_col])), row[label_col]) for _, row in df.iterrows()]    

In [95]:
import gradio as gr


with gr.Blocks() as demo:
    with gr.Row():
        gallery = gr.Gallery(
                label="Found images", show_label=False, elem_id="gallery"
            ).style(columns=[3], rows=[3], object_fit="contain", height="auto")
    with gr.Row():
        with gr.Tab("Embeddings"):
            vector_query = gr.Textbox(value="people riding bikes", show_label=False)
            gr.Button("Submit").click(find_image_vectors, inputs=vector_query, outputs=gallery)
        with gr.Tab("Keywords"):
            keyword_query = gr.Textbox(value="turtle", show_label=False)
            gr.Button("Submit").click(find_image_keywords, inputs=keyword_query, outputs=gallery)
        with gr.Tab("SQL"):
            sql_query = gr.Textbox(value="SELECT * from diffusiondb WHERE title like '%encoder%' LIMIT 9", show_label=False)
            gr.Button("Submit").click(find_image_sql, inputs=sql_query, outputs=gallery)
        
demo.launch()

Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.
